# Temperature Reading of PMICs (all switchers) and FPGA Core
---

## Aim

* This notebook illustrates how to measure the PMIC switchers' temperature and the FPGA's core temperature to observe what impact your application is having on the power sources' temperatures and FPGA core.

## Revision History

* Initial Release

---

## Load _base_ Overlay

In [1]:
from pynq.overlays.base import BaseOverlay
base=BaseOverlay('base.bit')

Load SMBus since PMIC devices are connected using SMBus through channel 3 of the I2C MUX

In [2]:
from smbus2 import *
bus = SMBus(3)

### Reading PMIC1 to PMIC3 Temperature

PMIC1 is addressed at 0x43, PMIC2 at 0x44, and PMIC3 at 0x46. Select each PMIC at a time, and read all five switchers of the PMIC.

In [3]:
for i in range(3):
    if i==0:
        address = 0x43
    elif i==1:
        address = 0x44
    else:
        address = 0x46
    for switcher in range(5):
        write1 = i2c_msg.write(address, [0x0, switcher])
        write = i2c_msg.write(address, [0x8D])
        read = i2c_msg.read(address, 2)
        with SMBus(3) as bus:
            bus.i2c_rdwr(write1)
            bus.i2c_rdwr(write, read)
            read_data=list(read)
            if switcher == 0:
                switcher_char = 'A'
            elif switcher == 1:
                switcher_char = 'B'
            elif switcher == 2:
                switcher_char = 'C'
            elif switcher == 3:
                switcher_char = 'D'
            else:
                switcher_char = 'LDO'        
            print('PMIC',i+1,'=Switcher',switcher_char,':', read_data[0], 'degree C')


PMIC 1 =Switcher A : 38 degree C
PMIC 1 =Switcher B : 38 degree C
PMIC 1 =Switcher C : 37 degree C
PMIC 1 =Switcher D : 37 degree C
PMIC 1 =Switcher LDO : 38 degree C
PMIC 2 =Switcher A : 43 degree C
PMIC 2 =Switcher B : 43 degree C
PMIC 2 =Switcher C : 44 degree C
PMIC 2 =Switcher D : 44 degree C
PMIC 2 =Switcher LDO : 44 degree C
PMIC 3 =Switcher A : 48 degree C
PMIC 3 =Switcher B : 48 degree C
PMIC 3 =Switcher C : 47 degree C
PMIC 3 =Switcher D : 47 degree C
PMIC 3 =Switcher LDO : 47 degree C


### Core temperature reading

FPGA core temperature can be read through the System Management IP of the FPGA. Instantiate the IP and define the required registers.

In [4]:
from time import sleep
xadc=base.system_management_wiz_0.mmio
# define registers offset, masks related to System Management IP
XSM_SRR_OFFSET=0
XSM_SRR_IPRST_MASK=0x0000000A
XSM_IP_OFFSET=0x200
XSM_CFR1_SEQ_VALID_MASK=0xF000
XSM_CFR1_SEQ_SAFEMODE_MASK=0x0000
XSM_TEMP_OFFSET=XSM_IP_OFFSET+0x200
XSM_CFR0_OFFSET=XSM_IP_OFFSET + 0x300
XSM_CFR1_OFFSET=XSM_IP_OFFSET + 0x304

Reset the IP and configure to measure the Core temperature.

In [5]:
# Reset the IP
xadc.write(XSM_SRR_OFFSET,XSM_SRR_IPRST_MASK)
# disable channel sequencer before configuring the sequence register
regvalue=xadc.read(XSM_CFR1_OFFSET)
xadc.write(XSM_CFR0_OFFSET, (regvalue & ~XSM_CFR1_SEQ_VALID_MASK | XSM_CFR1_SEQ_SAFEMODE_MASK))

Measure every 2 seconds for five times and display the result.

In [6]:
# Read the current value of the on-chip temperature and VCCAUX voltage
for i in range(5):
    print("Core temperature: "+"{:.2f}".format(((xadc.read(XSM_TEMP_OFFSET))/65536)/0.00199451786-273.67))
    sleep(2)

Core temperature: 32.60
Core temperature: 33.52
Core temperature: 32.10
Core temperature: 31.38
Core temperature: 33.32


Copyright (C) 2021 Xilinx, Inc

---

---